In [1]:
import pandas as pd
import pickle
import requests
import numpy as np
import sys
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import random
from scipy.spatial.distance import cosine
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
# from chembl_webresource_client.new_client import new_client
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, Reshape, GRU, SpatialDropout1D, LSTM, Dropout
from keras.layers import BatchNormalization, MaxPool1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

/home/kavya/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/kavya/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/kavya/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/kavya/anaconda3/lib/python3.6/site-packages/

In [5]:
df = pd.read_csv("../data/drug_class_identification/phase1/drugbank.csv")
df.shape
df.drop(df[df.smiles.isna()].index, inplace = True)
df.drop(df[df.atc.isna()].index, inplace = True)
df.drop(df[[False if len(smile)<250 else True for smile in df.smiles]].index, inplace = True)

In [6]:
df['atc1'] = [a[0] for a in df['atc']]

In [7]:
df.head()

,name,atc,inchi_key,smiles,atc1
13,goserelin,L02AE03,BLCLNMBMMGCOAS-URPVMXJPSA-N,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,L
33,desmopressin,H01BA02,NFLWUMRGJYTJIN-NXBWRCJVSA-N,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...,H
47,cetrorelix,H01CC02,SBNPWPIBESPSIF-MHWMIDJBSA-N,CC(C)C[C@H](NC(=O)[C@@H](CCCNC(N)=O)NC(=O)[C@H...,H
84,ciclosporin,L04AD01,PMATZTZNYRCHOR-CGLBZJNRSA-N,CC[C@@H]1NC(=O)[C@H]([C@H](O)[C@H](C)C\C=C\C)N...,L
95,octreotide,H01CB02,DEQANNDTNATYII-OULOTJBUSA-N,[H][C@]1(NC(=O)[C@H](CCCCN)NC(=O)[C@@H](CC2=CN...,H


In [8]:
df.describe()

,name,atc,inchi_key,smiles,atc1
count,2737,2737,2737,2737,2737
unique,2737,2640,2737,2737,14
top,dimethylaminopropionylphenothiazine,J01CR50,HBGOLJKPSFNJSD-UHFFFAOYSA-N,COC1=CC2=C(C=CC=C2CCNC(C)=O)C=C1,N
freq,1,22,1,1,469


In [9]:
df['smiles'] = [smile.ljust(250, '*') for smile in df['smiles']] 

### One Hot Encoding

In [10]:
vocabulary = set()
allSmiles = ""
for smile in df['smiles']:
    allSmiles+=smile
    for char in smile:
        vocabulary.add(char)
freq = Counter(allSmiles).most_common(25)

In [11]:
vocabulary = [a for a,b in freq]

In [18]:
'*' in vocabulary

True

In [57]:
oe = np.asarray(pd.get_dummies(np.asarray(vocabulary)))
oe_dict = {}
for a in range(len(vocabulary)):
    oe_dict[vocabulary[a]] = oe[a]

In [58]:
def encodeSmile(smile, oe):
    word = []
    for char in smile:
        try:
            temp = list(oe[char])
        except:
            temp = [0 for a in range(len(vocabulary))]
        word = word + temp
    return np.asarray(word, dtype=int)

In [59]:
X = []
X = [encodeSmile(smile, oe_dict) for smile in df['smiles']]

In [60]:
X = np.asarray(X)

In [61]:
ATCclasssubset = ['A', 'J', 'N', 'C']
num_classes = len(ATCclasssubset)

X_df = pd.DataFrame(X)
X_df['target'] = df['atc1']
X_df = X_df[X_df['target'].isin(ATCclasssubset)]
g = X_df.groupby('target')
X_df = g.apply(lambda x: x.sample(g.size().min())).reset_index(drop=True)

X = X_df[X_df.columns[:-1]]

### Prepare Data

In [62]:
y = pd.get_dummies(np.asarray(X_df['target']))                

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [68]:
temp = [[0 for b in range(0, num_classes)] for a in range(0, num_classes)]
for a in range(0, num_classes):
    temp[a][a] = 1
temp = np.asarray(temp)
atc_classes = y.columns

### Helper function

In [69]:
def accuracy(y_true, y_pred, atc_classes=atc_classes):
    print("Overall Accuracy:", np.sum(np.equal(y_true, y_pred))/y_true.shape[0])
    matrix = confusion_matrix(y_true, y_pred)
    print(classification_report(y_true, y_pred))
    plt.figure(figsize=(10, 10))
    sns.heatmap(matrix,annot=True,cbar=False, xticklabels=atc_classes, yticklabels=atc_classes)

### Make the network

In [72]:
class Network:
    def __init__(self, num_classes):
        self.model = Sequential()
        self.model.add(Reshape((250, 25), input_shape=(6250, )))
        self.model.add(Conv1D(192, 5, activation='relu', input_shape=(250, 25)))
        self.model.add(MaxPool1D(5))
        self.model.add(BatchNormalization())        
#         self.model.add(Conv1D(192, 5, activation='relu'))
        self.model.add(Conv1D(140, 3, activation='relu'))
#         self.model.add(LSTM(70, dropout=0.7, recurrent_dropout=0.2, activation='selu', return_sequences=True,))
#         self.model.add(LSTM(70, dropout=0.4, recurrent_dropout=0.2, activation='relu'))
        self.model.add(Flatten())
#         self.model.add(Dense(100, activation='relu'))
        self.model.add(Dropout(rate=0.4))
        self.model.add(Dense(num_classes, activation='softmax'))
        self.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        print(self.model.summary())
        
    def train(self, X, y, X_test, y_test, epochs=20):
        self.model.fit(X, y, epochs=epochs, validation_data=(X_test, y_test))
    
    def test(self, X, y):
        y_pred = self.model.predict(X).argmax(axis=1)
        y_true = y.argmax(axis=1)
        accuracy(y_true, y_pred)
        

In [73]:
network = Network(num_classes)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_2 (Reshape)          (None, 250, 25)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 246, 192)          24192     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 49, 192)           0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 49, 192)           768       
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 47, 140)           80780     
_________________________________________________________________
flatten_2 (Flatten)          (None, 6580)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 6580)              0         
__________

In [74]:
network.train(X_train, y_train, X_test, y_test, epochs=10)

Instructions for updating:
Use tf.cast instead.
Train on 324 samples, validate on 160 samples
Epoch 1/10
324/324 [==============================] - 3s 8ms/step - loss: 1.9941 - acc: 0.2593 - val_loss: 1.6420 - val_acc: 0.3312
Epoch 2/10
324/324 [==============================] - 1s 2ms/step - loss: 1.3769 - acc: 0.4012 - val_loss: 1.5095 - val_acc: 0.2750
Epoch 3/10
324/324 [==============================] - 1s 3ms/step - loss: 1.1035 - acc: 0.4969 - val_loss: 1.5113 - val_acc: 0.2938
Epoch 4/10
324/324 [==============================] - 1s 2ms/step - loss: 0.8458 - acc: 0.6821 - val_loss: 1.4975 - val_acc: 0.3625
Epoch 5/10
324/324 [==============================] - 1s 2ms/step - loss: 0.6824 - acc: 0.7654 - val_loss: 1.7298 - val_acc: 0.2500
Epoch 6/10
324/324 [==============================] - 1s 2ms/step - loss: 0.5748 - acc: 0.8148 - val_loss: 1.6902 - val_acc: 0.3500
Epoch 7/10
324/324 [==============================] - 1s 2ms/step - loss: 0.5406 - acc: 0.8117 - val_loss: 1.7064 

In [75]:
s = network.test(X_test, y_test)

AttributeError: 'DataFrame' object has no attribute 'argmax'

In [293]:
y_test

array([[0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],


In [43]:
oeAtc = pd.get_dummies(np.asarray(X_df['target']))

In [44]:
oeAtc

,A,C,J,N
0,1,0,0,0
1,1,0,0,0
2,1,0,0,0
3,1,0,0,0
4,1,0,0,0
...,...,...,...,...
479,0,0,0,1
480,0,0,0,1
481,0,0,0,1
482,0,0,0,1


In [45]:
oe

,#,(,),*,+,-,.,/,1,2,...,F,H,I,N,O,S,[,\,],l
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
6,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
